In [30]:
from os import listdir
from os.path import join
import regex as re
import numpy as np
from random import randint


def splitContentBySections(content: str):
    sections = [{'content': '# ' + section} for section in filter(None, re.split("\n##(?=[^#])", content))]
    
    for section in sections:
        section['labels'] = labelSection(section['content'])
        
    sections[0]['labels'].append('first')
        
    return sections

def findRegexHeadings(header):
    return f'(#+\s*(?:\d*|[ivx]*)\.?\s*{header})'

def getGenericHeadingCheckerFunction(*args):    
    return (lambda s: np.array([bool(re.findall(findRegexHeadings(header),s)) for header in args]).any())
    

def labelSection(content: str):    
    mappings = {
                        'reference': getGenericHeadingCheckerFunction('references', 'citations'),
                        'method': getGenericHeadingCheckerFunction('methodology', 'method', 'approach', 'experiment'),
                        'abstract': getGenericHeadingCheckerFunction('abstract'),
                        'appendix': getGenericHeadingCheckerFunction('appendix'),
                        'background': getGenericHeadingCheckerFunction('introduction', 'related work', 'background'),
                        'conclusion': getGenericHeadingCheckerFunction('conclusion', 'discussion'),
                        'results': getGenericHeadingCheckerFunction('results')
    }
    
    labels = [label for label, func in mappings.items() if func(content)]
    return labels



In [38]:
markdown_path = '../data/Markdown'
all_markdowns = listdir(markdown_path)

num = randint(1, len(all_markdowns)-1)
print(num)
sample = all_markdowns[num]
print(sample)
with open(join(markdown_path, sample), 'r') as f:
    content = f.read().lower()
sections = splitContentBySections(content)
sections


1401
32ac52069e562d4f900afee70bdca63f53461481.mmd


[{'content': '# # qlora: efficient finetuning of quantized llms\n\n tim dettmers\n\nequal contribution.\n\nartidoro pagnoni\n\nequal contribution.\n\nari holtzman\n\nluke zettlemoyer\n\nuniversity of washington\n\n{dettmers,artidoro,ahai,lsz}@cs.washington.edu\n\n###### abstract\n\nwe present qlora, an efficient finetuning approach that reduces memory usage enough to finetune a 65b parameter model on a single 48gb gpu while preserving full 16-bit finetuning task performance. qlora backpropagates gradients through a frozen, 4-bit quantized pretrained language model into low rank adapters (lora). our best model family, which we name **guanaco**, outperforms all previous openly released models on the vicuna benchmark, reaching 99.3% of the performance level of chatgpt while only requiring 24 hours of finetuning on a single gpu. qlora introduces a number of innovations to save memory without sacrificing performance: (a) 4-bit normalfloat (nf4), a new data type that is information theoretic

In [ ]:
markdown_sections = []
percentages = []
for markdown in all_markdowns:
    with open(join(markdown_path, markdown), 'r') as f:
        content = f.read().lower()
    
    sections = splitContentBySections(content)
    has_method = len([sec for sec in sections if 'method' in sec['labels']]) > 0
    markdown_sections.append((sections, has_method))
    
    if has_method:
        method_length = np.array([len(section['content']) for section in sections if 'method' in section['labels']]).sum()
        all_length = np.array([len(section['content']) for section in sections]).sum()
        percentages.append(method_length/all_length)
        
print(f"Average content percent as method: {np.array(percentages).mean()}")


In [ ]:
len([sections for sections, has_method in markdown_sections if not has_method])/len(all_markdowns)

i += 1
[sections for sections, has_method in markdown_sections if not has_method][i]